In [1]:
import ibis
from collections import defaultdict, deque

def join_sort(joins):
    # Create a graph to represent dependencies between tables
    graph = defaultdict(list)
    indegree = defaultdict(int)

    # Build the graph and compute indegree
    for join in joins:
        for left_table_col, right_table_col in join.items():
            left_table, left_col = left_table_col.split('.')
            right_table, right_col = right_table_col.split('.')
            graph[left_table].append((right_table, join))
            indegree[right_table] += 1
            if left_table not in indegree:
                indegree[left_table] = 0

    # Use a deque to perform stable topological sorting
    queue = deque(sorted([table for table in indegree if indegree[table] == 0]))
    sorted_joins = []

    while queue:
        table = queue.popleft()
        for neighbor, join in sorted(graph[table], key=lambda x: x[0]):
            sorted_joins.append(join)
            indegree[neighbor] -= 1
            if indegree[neighbor] == 0:
                queue.append(neighbor)
                queue = deque(sorted(queue))  # Maintain stable order

    return sorted_joins

def gvq_to_ibis(json_data):
    # Convert JSON to tables and joins
    tables = [node['label'] for node in json_data['nodes']]
    joins = [{edge['label'].split(' = ')[0]: edge['label'].split(' = ')[1]} for edge in json_data['edges']]
    print('>>joins:', joins)
    
    # Sort the joins based on dependencies
    sorted_joins = join_sort(joins)
    print('>>sorted_joins:', sorted_joins)
    
    # Connect to the DuckDB database
    con = ibis.connect("duckdb://")
    con.attach_sqlite("Chinook_Sqlite.sqlite")
    print('list_table:', con.list_tables())
    
    # Load tables into Ibis
    ibis_tables = {table: con.table(table) for table in tables}

    # Initialize a dictionary to hold the renamed columns for each table
    renamed_columns = {table: {} for table in tables}
    
    # Rename columns to unique names
    for table in tables:
        rename_dict = {f"{table}_{col}": col for col in ibis_tables[table].columns}
        renamed_columns[table] = {col: f"{table}_{col}" for col in ibis_tables[table].columns}
        ibis_tables[table] = ibis_tables[table].rename(rename_dict)
        # Debugging: Print the renamed columns to verify
        print(f"Renamed columns for table {table}: {ibis_tables[table].columns}")

    # Update join conditions to use renamed columns
    updated_joins = []
    for join in sorted_joins:
        updated_join = {}
        for left_table_col, right_table_col in join.items():
            left_table, left_col = left_table_col.split('.')
            right_table, right_col = right_table_col.split('.')
            try:
                updated_join[f"{renamed_columns[left_table][left_col]}"] = f"{renamed_columns[right_table][right_col]}"
            except KeyError as e:
                print(f"KeyError: {e} in join condition {join}")
                continue
        if updated_join:
            updated_joins.append(updated_join)
        # Debugging: Print the updated join conditions
        print(f"Updated join: {updated_join}")

    # Initialize a variable to hold the join expression
    join_expr = None

    for idx, join in enumerate(updated_joins):
        left_table_col, right_table_col = list(join.items())[0]
        left_table = left_table_col.split('_')[0]
        right_table = right_table_col.split('_')[0]

        if join_expr is None:
            # First join
            join_expr = ibis_tables[left_table].join(
                ibis_tables[right_table],
                ibis_tables[left_table][left_table_col] == ibis_tables[right_table][right_table_col],
                how='left'
            )
        else:
            # Subsequent joins using the previous join result
            join_expr = join_expr.join(
                ibis_tables[right_table],
                join_expr[left_table_col] == ibis_tables[right_table][right_table_col],
                how='left'
            )

    return join_expr

# Example usage:
# Test Case 5 with custom join order
json_data5 = {
  "nodes": [
    { "id": "1", "label": "Customer", "position": { "x": 100, "y": 100 } },
    { "id": "2", "label": "Invoice", "position": { "x": 300, "y": 100 } },
    { "id": "3", "label": "InvoiceLine", "position": { "x": 500, "y": 100 } },
    { "id": "4", "label": "Track", "position": { "x": 700, "y": 100 } },
    { "id": "5", "label": "Employee", "position": { "x": 900, "y": 100 } }
  ],
  "edges": [
    { "source": "1", "target": "2", "label": "Customer.CustomerId = Invoice.CustomerId" },
    { "source": "3", "target": "4", "label": "InvoiceLine.TrackId = Track.TrackId" },
    { "source": "2", "target": "3", "label": "Invoice.InvoiceId = InvoiceLine.InvoiceId" },
    { "source": "1", "target": "5", "label": "Customer.SupportRepId = Employee.EmployeeId" }
  ]
}


json_data = json_data5

# Convert GVQ to Ibis expression
ibis_expr = gvq_to_ibis(json_data)
print(ibis_expr)

# Execute the Ibis expression and fetch results
result = ibis_expr.execute()
result.info()

>>joins: [{'Customer.CustomerId': 'Invoice.CustomerId'}, {'InvoiceLine.TrackId': 'Track.TrackId'}, {'Invoice.InvoiceId': 'InvoiceLine.InvoiceId'}, {'Customer.SupportRepId': 'Employee.EmployeeId'}]
>>sorted_joins: [{'Customer.SupportRepId': 'Employee.EmployeeId'}, {'Customer.CustomerId': 'Invoice.CustomerId'}, {'Invoice.InvoiceId': 'InvoiceLine.InvoiceId'}, {'InvoiceLine.TrackId': 'Track.TrackId'}]
list_table: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Renamed columns for table Customer: ['Customer_CustomerId', 'Customer_FirstName', 'Customer_LastName', 'Customer_Company', 'Customer_Address', 'Customer_City', 'Customer_State', 'Customer_Country', 'Customer_PostalCode', 'Customer_Phone', 'Customer_Fax', 'Customer_Email', 'Customer_SupportRepId']
Renamed columns for table Invoice: ['Invoice_InvoiceId', 'Invoice_CustomerId', 'Invoice_InvoiceDate', 'Invoice_BillingAddress', 'Invoice_BillingCity', 'Invoice

In [2]:
json_data6={
  "nodes": [
    { "id": "1", "label": "Customer", "position": { "x": 100, "y": 100 } },
    { "id": "2", "label": "Invoice", "position": { "x": 300, "y": 100 } },
    { "id": "3", "label": "InvoiceLine", "position": { "x": 500, "y": 100 } },
    { "id": "4", "label": "Track", "position": { "x": 700, "y": 100 } },
    { "id": "5", "label": "Album", "position": { "x": 900, "y": 100 } },
    { "id": "6", "label": "Artist", "position": { "x": 1100, "y": 100 } },
    { "id": "7", "label": "Genre", "position": { "x": 1300, "y": 100 } },
    { "id": "8", "label": "Employee", "position": { "x": 1500, "y": 100 } }
  ],
  "edges": [
    { "source": "1", "target": "2", "label": "Customer.CustomerId = Invoice.CustomerId" },
    { "source": "2", "target": "3", "label": "Invoice.InvoiceId = InvoiceLine.InvoiceId" },
    { "source": "3", "target": "4", "label": "InvoiceLine.TrackId = Track.TrackId" },
    { "source": "4", "target": "5", "label": "Track.AlbumId = Album.AlbumId" },
    { "source": "5", "target": "6", "label": "Album.ArtistId = Artist.ArtistId" },
    { "source": "4", "target": "7", "label": "Track.GenreId = Genre.GenreId" },
    { "source": "1", "target": "8", "label": "Customer.SupportRepId = Employee.EmployeeId" }
  ]
}



json_data = json_data6

# Convert GVQ to Ibis expression
ibis_expr = gvq_to_ibis(json_data)
print(ibis_expr)

# Execute the Ibis expression and fetch results
result = ibis_expr.execute()
result.info()

>>joins: [{'Customer.CustomerId': 'Invoice.CustomerId'}, {'Invoice.InvoiceId': 'InvoiceLine.InvoiceId'}, {'InvoiceLine.TrackId': 'Track.TrackId'}, {'Track.AlbumId': 'Album.AlbumId'}, {'Album.ArtistId': 'Artist.ArtistId'}, {'Track.GenreId': 'Genre.GenreId'}, {'Customer.SupportRepId': 'Employee.EmployeeId'}]
>>sorted_joins: [{'Customer.SupportRepId': 'Employee.EmployeeId'}, {'Customer.CustomerId': 'Invoice.CustomerId'}, {'Invoice.InvoiceId': 'InvoiceLine.InvoiceId'}, {'InvoiceLine.TrackId': 'Track.TrackId'}, {'Track.AlbumId': 'Album.AlbumId'}, {'Track.GenreId': 'Genre.GenreId'}, {'Album.ArtistId': 'Artist.ArtistId'}]
list_table: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Renamed columns for table Customer: ['Customer_CustomerId', 'Customer_FirstName', 'Customer_LastName', 'Customer_Company', 'Customer_Address', 'Customer_City', 'Customer_State', 'Customer_Country', 'Customer_PostalCode', 'Customer_Phon

In [3]:
json_data7 = {
  "nodes": [
    { "id": "1", "label": "Playlist", "position": { "x": 100, "y": 100 } },
    { "id": "2", "label": "PlaylistTrack", "position": { "x": 300, "y": 100 } },
    { "id": "3", "label": "Track", "position": { "x": 500, "y": 100 } },
    { "id": "4", "label": "Album", "position": { "x": 700, "y": 100 } },
    { "id": "5", "label": "Artist", "position": { "x": 900, "y": 100 } },
    { "id": "6", "label": "Genre", "position": { "x": 1100, "y": 100 } },
    { "id": "7", "label": "MediaType", "position": { "x": 1300, "y": 100 } },
    { "id": "8", "label": "Customer", "position": { "x": 1500, "y": 100 } },
    { "id": "9", "label": "Invoice", "position": { "x": 1700, "y": 100 } },
    { "id": "10", "label": "InvoiceLine", "position": { "x": 1900, "y": 100 } }
  ],
  "edges": [
    # { "source": "1", "target": "2", "label": "Playlist.PlaylistId = PlaylistTrack.PlaylistId" },
    # { "source": "2", "target": "3", "label": "PlaylistTrack.TrackId = Track.TrackId" },
    { "source": "3", "target": "4", "label": "Track.AlbumId = Album.AlbumId" },
    { "source": "4", "target": "5", "label": "Album.ArtistId = Artist.ArtistId" },
    { "source": "3", "target": "6", "label": "Track.GenreId = Genre.GenreId" },
    { "source": "3", "target": "7", "label": "Track.MediaTypeId = MediaType.MediaTypeId" },
    { "source": "8", "target": "9", "label": "Customer.CustomerId = Invoice.CustomerId" },
    { "source": "9", "target": "10", "label": "Invoice.InvoiceId = InvoiceLine.InvoiceId" },
    { "source": "10", "target": "3", "label": "InvoiceLine.TrackId = Track.TrackId" }
  ]
}


json_data = json_data7

# Convert GVQ to Ibis expression
ibis_expr = gvq_to_ibis(json_data)
print(ibis_expr)

# Execute the Ibis expression and fetch results
result = ibis_expr.execute()
result.info()

>>joins: [{'Track.AlbumId': 'Album.AlbumId'}, {'Album.ArtistId': 'Artist.ArtistId'}, {'Track.GenreId': 'Genre.GenreId'}, {'Track.MediaTypeId': 'MediaType.MediaTypeId'}, {'Customer.CustomerId': 'Invoice.CustomerId'}, {'Invoice.InvoiceId': 'InvoiceLine.InvoiceId'}, {'InvoiceLine.TrackId': 'Track.TrackId'}]
>>sorted_joins: [{'Customer.CustomerId': 'Invoice.CustomerId'}, {'Invoice.InvoiceId': 'InvoiceLine.InvoiceId'}, {'InvoiceLine.TrackId': 'Track.TrackId'}, {'Track.AlbumId': 'Album.AlbumId'}, {'Track.GenreId': 'Genre.GenreId'}, {'Track.MediaTypeId': 'MediaType.MediaTypeId'}, {'Album.ArtistId': 'Artist.ArtistId'}]
list_table: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Renamed columns for table Playlist: ['Playlist_PlaylistId', 'Playlist_Name']
Renamed columns for table PlaylistTrack: ['PlaylistTrack_PlaylistId', 'PlaylistTrack_TrackId']
Renamed columns for table Track: ['Track_TrackId', 'Track_Name', 'T